In [ ]:
from pyspark.sql import SQLContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
import utils
%matplotlib inline

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.load('file:/home/cloudera/epri/minute_weather.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

In [ ]:
# count rows



In [ ]:
# filter rows



In [ ]:
# calculate summary statistics



In [ ]:
# drop rows with nulls



In [ ]:
featuresUsed = ['air_pressure', 'air_temp', 'avg_wind_direction', 'avg_wind_speed', 'max_wind_direction', 
        'max_wind_speed','relative_humidity']
assembler = VectorAssembler(inputCols=featuresUsed, outputCol="features_unscaled")
assembled = assembler.transform(workingDF)

In [ ]:
scaler = StandardScaler(inputCol="features_unscaled", outputCol="features", withStd=True, withMean=True)
scalerModel = scaler.fit(assembled)
scaledData = scalerModel.transform(assembled)

In [ ]:
scaledData = scaledData.select("features", "rowID")

elbowset = scaledData.filter((scaledData.rowID % 3) == 0).select("features")
elbowset.persist()

In [ ]:
# generate clusters for elbow plot



In [ ]:
# show elbow plot



In [ ]:
scaledDataFeat = scaledData.select("features")
scaledDataFeat.persist()

In [ ]:
kmeans = KMeans(k=12, seed=1)
model = kmeans.fit(scaledDataFeat)
transformed = model.transform(scaledDataFeat)

In [ ]:
centers = model.clusterCenters()
centers

In [ ]:
P = utils.pd_centers(featuresUsed, centers)

## Dry Days

## Warm Days

## Cool Days

In [ ]:
utils.parallel_plot(P[(P['relative_humidity'] > 0.5) & (P['air_temp'] < 0.5)], P)

## Other Days

In [ ]:
utils.parallel_plot(P.iloc[[2]], P)